In [1]:
import numpy as np
import pandas as pd

## FUNCTIONS

In [2]:
def transform_parameters (parameters, transformation = "quantile"):
    
    """
    This function is used to normalize the parameters.
    @ parameters: pd.DataFrame of parameters to normalize
    @ transformation: Type of normalization applied i.e. quantile, standard...
    """
    
    columns = parameters.columns
    
    from sklearn.preprocessing import QuantileTransformer, StandardScaler
    import pandas as pd
    
    if transformation == "quantile":
        
        t = QuantileTransformer()
        trans_params =  t.fit_transform(parameters)
        
    if transformation == "standard":
        
        t = StandardScaler()
        trans_params =  t.fit_transform(parameters)
    
    return pd.DataFrame(trans_params, columns=columns), t

In [3]:
def antitransform_parameters (parameters, transformer):
    
    """
    This function is used to normalize the parameters.
    @ parameters: pd.DataFrame of normalized parameters
    @ transformer: Transformer used for normalization
    """  
    
    columns = parameters.columns

    return pd.DataFrame(transformer.inverse_transform(parameters), columns=columns)

In [4]:
def normalize_bands (frequencies, num_bands = 5, num_k_points  = 31, transformation = "quantile"):
    
    """
    This function is used to normalize the bands individually.
    @ frequencies: pd.DataFrame of simulated frequencies per k-points
    @ num_bands: Number of bands in simulations
    @ num_k_points: Number of k-points in simulations
    @ transformation: Type of normalization applied i.e. quantile, standard...
    """
    
    from sklearn.preprocessing import QuantileTransformer, StandardScaler
    import pandas as pd
    
    
    for i in range(int(num_bands)):
        
        column_start = "Band_"+str(i)+"_k_0"
        column_end = "Band_"+str(i)+"_k_"+str(num_k_points-1)
    
        df_band = frequencies.loc[:, column_start : column_end]
        columns = df_band.columns
        
        if transformation == "quantile":
        
            t = QuantileTransformer()
            trans_band =  t.fit_transform(df_band)
            trans_band = pd.DataFrame(df_band, columns=columns)
        
        if transformation == "standard":
        
            t = StandardScaler()
            trans_band =  t.fit_transform(df_band) 
            trans_band = pd.DataFrame(df_band, columns=columns)
            
        if i == 0:
            
            transformed_bands = trans_band.copy()
            
        else:
            
            transformed_bands = pd.concat([transformed_bands, trans_band], axis=1)
    
    return transformed_bands

## EXAMPLE OF USE:

### Transformation of parameters

In [5]:
# Uniform choice of 50 parameters in defined ranges

n = np.random.uniform(low=2, high=18, size=(50,))
e = np.random.uniform(low=0.1, high=1.0, size=(50,))
r = np.random.uniform(low=10, high=130, size=(50,)) #in nanometers
t = np.random.uniform(low=0, high=90, size=(50,)) # in º

In [6]:
params_df = pd.DataFrame(np.vstack([n,e,r,t]).T, columns = ["n","e","r","t"])

In [7]:
params_df.head(10)

,n,e,r,t
0,3.266034,0.832999,49.775630,45.956340
1,15.107270,0.741070,58.452203,2.816590
2,10.605189,0.909960,57.353053,67.561321
3,6.802802,0.864273,41.595509,71.518977
4,14.742720,0.862864,91.538197,31.557425
5,3.494111,0.879439,89.691053,71.502613
6,8.583366,0.117720,46.394640,76.367479
7,14.797969,0.246371,32.265925,10.128399
8,6.990195,0.540918,115.791596,27.055258
9,2.325915,0.343109,64.708934,49.210556


In [8]:
trans_params_df, transformer = transform_parameters (params_df, transformation = "standard")

You can also check that it works with the QuantileTransformer()

In [9]:
trans_params_df.head(10)

,n,e,r,t
0,-1.036722,0.907504,-0.542877,-0.048159
1,1.461303,0.580246,-0.262693,-1.798637
2,0.511545,1.181476,-0.298187,0.828505
3,-0.290606,1.018834,-0.807030,0.989094
4,1.384397,1.013819,0.805723,-0.632422
5,-0.988607,1.072825,0.746075,0.988430
6,0.085021,-1.638819,-0.652056,1.185831
7,1.396053,-1.180835,-1.108302,-1.501946
8,-0.251074,-0.132277,1.588916,-0.815107
9,-1.235050,-0.836456,-0.060650,0.083887


In [10]:
transformer

StandardScaler()

### Antitransformation of  parameters

In [11]:
antitrans_params = antitransform_parameters (trans_params_df, transformer)

In [12]:
antitrans_params.head(10)

,n,e,r,t
0,3.266034,0.832999,49.775630,45.956340
1,15.107270,0.741070,58.452203,2.816590
2,10.605189,0.909960,57.353053,67.561321
3,6.802802,0.864273,41.595509,71.518977
4,14.742720,0.862864,91.538197,31.557425
5,3.494111,0.879439,89.691053,71.502613
6,8.583366,0.117720,46.394640,76.367479
7,14.797969,0.246371,32.265925,10.128399
8,6.990195,0.540918,115.791596,27.055258
9,2.325915,0.343109,64.708934,49.210556


As we can see **we get the same result** as we had in the beginning!

###  Band normalization

In [13]:
frequencies = pd.read_csv('file_reader/frequencies_data.csv')  

We test a sample with 3239 simulations!

In [14]:
frequencies.head(10)

,Band_0_k_0,Band_0_k_1,Band_0_k_2,Band_0_k_3,Band_0_k_4,Band_0_k_5,Band_0_k_6,Band_0_k_7,Band_0_k_8,Band_0_k_9,...,Band_4_k_21,Band_4_k_22,Band_4_k_23,Band_4_k_24,Band_4_k_25,Band_4_k_26,Band_4_k_27,Band_4_k_28,Band_4_k_29,Band_4_k_30
0,0.0,0.25910,0.51819,0.77729,1.03638,1.29547,1.55455,1.81363,2.07270,2.33174,...,7.84786,7.52634,7.20901,6.89646,6.58935,6.28850,6.02166,5.73665,5.46049,5.21749
1,0.0,0.29122,0.58217,0.87254,1.16198,1.45004,1.73600,2.01853,2.29418,2.54952,...,8.00736,7.73822,7.46038,7.17809,6.89442,6.78260,6.75986,6.52295,6.29366,6.17325
2,0.0,0.28548,0.57076,0.85561,1.13972,1.42261,1.70340,1.98027,2.24829,2.48946,...,7.99410,7.73389,7.46315,7.18395,6.89933,6.61304,6.72267,6.48437,6.26683,6.16810
3,0.0,0.29789,0.59544,0.89228,1.18795,1.48188,1.77313,2.05991,2.33765,2.58941,...,8.08047,7.81203,7.53079,7.24485,6.95887,6.97181,6.93343,6.68770,6.46084,6.34544
4,0.0,0.26866,0.53727,0.80578,1.07412,1.34220,1.60987,1.87683,2.14231,2.40270,...,7.88221,7.56867,7.25482,6.94395,6.63799,6.34148,6.33800,6.05831,5.79489,5.62800
5,0.0,0.26300,0.52598,0.78894,1.05184,1.31467,1.57739,1.83992,2.10210,2.36311,...,7.85503,7.53406,7.21680,6.90416,6.59693,6.29608,6.16102,5.88639,5.61729,5.43269
6,0.0,0.26534,0.53066,0.79591,1.06107,1.32609,1.59093,1.85546,2.11944,2.38170,...,7.87126,7.55493,7.23970,6.92807,6.62157,6.32299,6.21483,5.95728,5.69801,5.53778
7,0.0,0.28707,0.57399,0.86060,1.14674,1.43223,1.71685,2.00030,2.28204,2.56019,...,7.97656,7.65788,7.34059,7.02677,6.71799,6.44952,6.80109,6.72492,6.48862,6.36634
8,0.0,0.26917,0.53828,0.80726,1.07604,1.34454,1.61261,1.88002,2.14615,2.40826,...,7.88375,7.57440,7.26306,6.95368,6.64853,6.35172,6.29046,6.07142,5.83180,5.69942
9,0.0,0.26883,0.53760,0.80624,1.07469,1.34284,1.61055,1.87754,2.14311,2.40406,...,7.88236,7.57198,7.25988,6.94980,6.64367,6.34293,6.26640,6.06141,5.82700,5.69938


We infere the number of k_points and bands from the columns (check last  column, indexing starts from 0):

In [15]:
num_bands = 5
num_k_points = 31

In [16]:
norm_bands = normalize_bands (frequencies, num_bands = 5, num_k_points  = 31,
                              transformation = "standard")

In [17]:
norm_bands

,Band_0_k_0,Band_0_k_1,Band_0_k_2,Band_0_k_3,Band_0_k_4,Band_0_k_5,Band_0_k_6,Band_0_k_7,Band_0_k_8,Band_0_k_9,...,Band_4_k_21,Band_4_k_22,Band_4_k_23,Band_4_k_24,Band_4_k_25,Band_4_k_26,Band_4_k_27,Band_4_k_28,Band_4_k_29,Band_4_k_30
0,0.0,0.25910,0.51819,0.77729,1.03638,1.29547,1.55455,1.81363,2.07270,2.33174,...,7.84786,7.52634,7.20901,6.89646,6.58935,6.28850,6.02166,5.73665,5.46049,5.21749
1,0.0,0.29122,0.58217,0.87254,1.16198,1.45004,1.73600,2.01853,2.29418,2.54952,...,8.00736,7.73822,7.46038,7.17809,6.89442,6.78260,6.75986,6.52295,6.29366,6.17325
2,0.0,0.28548,0.57076,0.85561,1.13972,1.42261,1.70340,1.98027,2.24829,2.48946,...,7.99410,7.73389,7.46315,7.18395,6.89933,6.61304,6.72267,6.48437,6.26683,6.16810
3,0.0,0.29789,0.59544,0.89228,1.18795,1.48188,1.77313,2.05991,2.33765,2.58941,...,8.08047,7.81203,7.53079,7.24485,6.95887,6.97181,6.93343,6.68770,6.46084,6.34544
4,0.0,0.26866,0.53727,0.80578,1.07412,1.34220,1.60987,1.87683,2.14231,2.40270,...,7.88221,7.56867,7.25482,6.94395,6.63799,6.34148,6.33800,6.05831,5.79489,5.62800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3234,0.0,0.29070,0.58113,0.87101,1.15996,1.44746,1.73262,2.01365,2.28590,2.53216,...,8.00143,7.72381,7.43841,7.15042,6.86885,6.73886,6.82793,6.57006,6.32272,6.17921
3235,0.0,0.26433,0.52863,0.79288,1.05705,1.32110,1.58498,1.84858,2.11167,2.37308,...,7.86401,7.54583,7.22964,6.91740,6.61026,6.30943,6.16410,5.92381,5.66819,5.51063
3236,0.0,0.29535,0.59038,0.88473,1.17800,1.46966,1.75890,2.04415,2.32147,2.57579,...,8.04332,7.77087,7.48963,7.20482,6.92180,6.94235,6.89541,6.64275,6.41654,6.30327
3237,0.0,0.26397,0.52792,0.79182,1.05565,1.31937,1.58292,1.84621,2.10899,2.37014,...,7.86309,7.54459,7.22820,6.91583,6.60857,6.30736,6.14820,5.91066,5.65532,5.49714


You can also check that it works with the QuantileTransformer()